<a href="https://colab.research.google.com/github/VarunV991/Hyperparameter-Tuning-Algorithms/blob/master/HyperOpt_HT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install hyperopt Library
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import datasets
from functools import partial
from hyperopt import hp,fmin,tpe,Trials
from hyperopt.pyll.base import scope

warnings.filterwarnings("ignore")

In [8]:
 #--------- Custom Dataset -----------#

iris = datasets.load_iris()
X = iris.data[:,:2]
y = iris.target

 #--------- Custom Dataset -----------#

## Using Hyperopt library

In [17]:
def optimize(params,x,y):

  #--------- Custom Code/Model -----------#

  model = RandomForestClassifier(**params)
  kf = StratifiedKFold(n_splits=5)

  #--------- Custom Code/Model -----------#

  # Cross Validation split
  accuracies = []
  for idx in kf.split(X=x,y=y):
    train_idx,val_idx = idx[0],idx[1]
    # Train
    Xtrain = x[train_idx]
    ytrain = y[train_idx]
    # Validation
    Xval = x[val_idx]
    yval = y[val_idx]

    model.fit(Xtrain,ytrain)
    pred = model.predict(Xval)
    fold_acc = accuracy_score(yval,pred)
    accuracies.append(fold_acc)
  
  return -1.0 * np.mean(accuracies)

In [18]:
param_space = {
    "max_depth":scope.int(hp.quniform("max_depth",3,15,1)),
    "n_estimators":scope.int(hp.quniform("n_estimators",100,600,100)),
    "criterion":hp.choice("criterion",["gini","entropy"]),
    "max_features":hp.uniform("max_features",0.01,1)
}

optimization_function = partial(optimize,x = X,y=y)
trials = Trials()

In [19]:
result = fmin(fn = optimization_function,
                     space=param_space,
                     max_evals=15,
                     trials=trials,
                      algo=tpe.suggest)

100%|██████████| 15/15 [00:38<00:00,  2.58s/it, best loss: -0.7866666666666667]


In [21]:
print(result)

{'criterion': 1, 'max_depth': 5.0, 'max_features': 0.6607198134304393, 'n_estimators': 400.0}


#### Here, for the categorical values, the output is the index of the optimal parameter in the param array